In [192]:
import shapefile
import json
import pandas as pd
import os
import zipfile
import codecs
import io

from bokeh.plotting import figure, output_file, show, hplot
from bokeh.models import Rect, HoverTool, ColumnDataSource

from collections import OrderedDict

In [58]:
def shapes_from_zip(shapezip):
    zipshape = zipfile.ZipFile(shapezip)
    ishp = [f for f in zipshape.namelist() if '.shp' in f][0]
    ishx = [f for f in zipshape.namelist() if '.shx' in f][0]
    idbf = [f for f in zipshape.namelist() if '.dbf' in f][0]
    partdic = {}
    for part in [ishp,ishx,idbf]:
        partdic[part] = io.BytesIO(zipshape.read(part))
    return shapefile.Reader(shp=partdic[ishp],shx=partdic[ishx],dbf=partdic[idbf])

In [181]:
def getParts ( shapeObj ):
    points = []
    num_parts = len( shapeObj.parts )
    end = len( shapeObj.points ) - 1
    segments = list( shapeObj.parts ) + [ end ]
    for i in range( num_parts ):
        points.append( shapeObj.points[ segments[i]:segments[i+1] ] )
    return points

def shapefile_to_dataframe(processed_shapefile):
    df = pd.DataFrame()
    shape_records = processed_shapefile.shapeRecords()
    shapes = [i.shape for i in shape_records]
    records = [i.record for i in shape_records]
    print(records[0])
    if str(records[0][1]) == '105051100':
        record_pos = 2
    else:
        record_pos = 1
    #shapes = [i.shape for i in shape_records]
    #shapes = pd.Series(shapes)
    locs = []
    rec = []
    for shapeObj,record in zip(shapes, records):
        points = []
        try:
            num_parts = len(shapeObj.parts)
        except:
            continue
        end = len(shapeObj.points) - 1
        segments = list(shapeObj.parts) + [end]
        for i in range(num_parts):
            points.append(shapeObj.points[ segments[i]:segments[i+1]])
        for point in points:
            locs.append(point)
            for_rec = record[record_pos]
            rec.append(for_rec)
    lat = []
    lng = []
    for loc in locs:
        lat.append([pair[0] for pair in loc])
        lng.append([pair[1] for pair in loc])
    df['lat'] = lat
    df['lng'] = lng
    df['name'] = rec
    return(df)

In [207]:
def plot_from_df(output_file_name, df, size):
    data_source = ColumnDataSource (
    data = dict(
            lat_list=adf['lat'],
            lng_list=adf['lng'],
            #color=colors['hex'],
            name=adf['name'],
            #data=colors[variable],
            #suburbs=colors['suburb'],
            #state=colors['state']
    )
    )
    output_file(output_file_name)
    #vaariable = '$'+variable
    TOOLS="pan,wheel_zoom,box_zoom,reset,hover,save"
    #if geo_filter == 'None':
    #    title_str = 'Australia'
    #else:
    #    title_str = geo_filter
    p = figure(#title=variable.capitalize()+" in "+title_str.capitalize()+" by Postcode", 
        plot_width = size, plot_height=size, tools=TOOLS)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.patches('lat_list',
              'lng_list',
              fill_color = 'red', fill_alpha=0.7,
              line_color = 'white', line_width = 0.5,
             source = data_source)
    hover = p.select(dict(type=HoverTool))
    hover.point_policy = "follow_mouse"
    hover.tooltips = OrderedDict([
            ("Name", "@name"),
            #(variable, '@data'),
            #("Suburbs",'@suburbs'),
            #("State",'@state'),
    ])
    show(p)

In [208]:
plot_from_df('output.html',adf, 1000)

In [189]:
ex1 = example.shapeRecord()

In [191]:
ex1.record

['1', '105051100', 'Botany Bay (C)', '11100', '110411100']

In [170]:
ex1 = example.shapeRecords()

In [172]:
ex2 = ex1[0].shape

In [173]:
ex2.parts

[0]

In [194]:
#example = shapes_from_zip('Shapefiles/2011_POA_shape.zip')
#example = shapes_from_zip('Shapefiles/1259030001_sla11aaust_shape.zip')
example = shapes_from_zip('Shapefiles/1259030001_ste11aaust_shape.zip')
adf = shapefile_to_dataframe(example)

['1', 'New South Wales']


In [195]:
adf

,lat,lng,name
0,"[149.916344, 149.916244, 149.9162, 149.9162070...","[-37.074006002000004, -37.0739880015, -37.0739...",New South Wales
1,"[150.074088, 150.074070016, 150.074076, 150.07...","[-36.421919001, -36.4219039975, -36.4218549910...",New South Wales
2,"[150.072756, 150.072734016, 150.072746016, 150...","[-36.4536030075, -36.453585007, -36.453560994,...",New South Wales
3,"[150.072810016, 150.072830016, 150.072652, 150...","[-36.448891002, -36.448956991500005, -36.44897...",New South Wales
4,"[150.074939008, 150.074936, 150.074938016, 150...","[-36.421544006, -36.4215230085, -36.421504009,...",New South Wales
5,"[150.072763008, 150.072751008, 150.072751008, ...","[-36.452820994, -36.452783994, -36.4527619975,...",New South Wales
6,"[150.137784, 150.137811008, 150.137744, 150.13...","[-36.305411995, -36.3054629995, -36.305523994,...",New South Wales
7,"[150.072140992, 150.072130016, 150.072134848, ...","[-36.4505310085, -36.450514007, -36.4505082165...",New South Wales
8,"[150.056199008, 150.056207008, 150.056202016, ...","[-36.4960559935, -36.4960159965, -36.495965991...",New South Wales
9,"[150.072966016, 150.072956992, 150.072963008, ...","[-36.4539489945, -36.453923002, -36.4539010055...",New South Wales


In [96]:
ex1 = example.shapeRecords()[0]

In [98]:
ex1.shape.points

[[151.2278672, -33.9560375915],
 [151.227534368, -33.9560964585],
 [151.226187488, -33.956335256500005],
 [151.226064736, -33.956357216],
 [151.225694272, -33.9564310125],
 [151.2255072, -33.9565256215],
 [151.225370208, -33.9565966245],
 [151.223690944, -33.9578598045],
 [151.223685472, -33.9578639115],
 [151.223634272, -33.957903631],
 [151.221929312, -33.9591936545],
 [151.22115232, -33.959096659000004],
 [151.221134336, -33.959184645],
 [151.221019328, -33.9597606795],
 [151.220954368, -33.9600586775],
 [151.220390432, -33.96298769],
 [151.220350272, -33.96318934],
 [151.22033008, -33.9632954375],
 [151.220310432, -33.963400684],
 [151.220259424, -33.963661682],
 [151.220163456, -33.964262691500004],
 [151.22017744, -33.964354692],
 [151.22079552, -33.968103717],
 [151.220898528, -33.968363716],
 [151.220655456, -33.96843335],
 [151.220636992, -33.9683969975],
 [151.220788992, -33.968345993],
 [151.220703008, -33.968155998],
 [151.220547008, -33.9682030065],
 [151.22008, -33.967204

In [52]:
ishape = r.shapes()[0]

ishape.points

[[150.851179008, -33.870690006],
 [150.851036992, -33.8705359935],
 [150.851167008, -33.8704520035],
 [150.851250016, -33.870541007],
 [150.851977504, -33.870069553],
 [150.852009632, -33.869905828],
 [150.85237900799999, -33.869654006],
 [150.85251500799998, -33.8689670085],
 [150.852652992, -33.868275996499996],
 [150.852708992, -33.867989006],
 [150.852726016, -33.8679059965],
 [150.853666816, -33.8631609315],
 [150.855872992, -33.863519998],
 [150.860520992, -33.8641570085],
 [150.863346016, -33.8645369985],
 [150.864156992, -33.863515002999996],
 [150.86450099200002, -33.863292004],
 [150.86383708800003, -33.86257672],
 [150.86429900800002, -33.8622780005],
 [150.864100992, -33.8620639925],
 [150.86678799999999, -33.8603189985],
 [150.86698601600003, -33.8605320075],
 [150.867804, -33.859999004],
 [150.86811299200002, -33.8592449995],
 [150.86824000000001, -33.858929001],
 [150.86855100800003, -33.858530992],
 [150.870011008, -33.857449001],
 [150.870746016, -33.856909004500004],


In [34]:
with zipfile.ZipFile('Data/1270055003_ssc_2011_aust_shape.zip') as izip:
    with izip.open('SSC_2011_AUST.shp','r') as ishp:
        with izip.open('SSC_2011_AUST.dbf','r') as idbf:
            locdic = shapefile.Reader(dbf=idbf, shp=ishp)



#locdic = shapefile.Reader(zip=shpzip)

#('Data/1270055003_ssc_2011_aust_shape.zip')

UnsupportedOperation: seek

In [12]:
zipfile.ZipFile.

TypeError: getinfo() missing 2 required positional arguments: 'self' and 'name'